## Comments

https://www.minfin.bg/bg/transparency/ <br>

- currently does not download MF_; NF files -> search for "@change files" to change


to do:
time.sleep change, double check scrapy



In [1]:
import sys
import os
import time

#!{sys.executable} -m pip install scrapy

from datetime import date,datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [11]:
path = r'/Users/bilyanavencheva/Desktop/All/DG_test' #r'/Users/bilyanavencheva/Desktop/All/DataForGood'

## create sub directories to store the different types of documents
# @change files
folders = ['/SEBRA']#['/SEBRA','/NF_SEBRA','/MF_SEBRA']

#where to start: start and end date are defined later (if data is already available in folders, below values are not used)
min_year = 2021
min_month = 5
min_day = 15

def get_urls(min_year = min_year, min_month = min_month, min_day = min_day):
    #max date always current date
    max_year = datetime.now().date().year
    years = [str(year) for year in range(min_year,max_year + 1)]

    for folder in folders:
        #if folder already exists, check if there are subfolders with years, create if necessary
        if os.path.exists(path + folder) == True:
            print ("Already existing directory: ", path + folder)
            for y in years:
                try:
                    os.mkdir(path + folder + '/' + y)
                except FileExistsError:
                    pass
        else:
            try:
                os.mkdir(path + folder)
                print ("Successfully created the directory: ", path + folder)
            except OSError:
                print ("Failure creation of the directory:", path + folder)

            for y in years:
                try:
                    os.mkdir(path + folder + '/' + y)
                except FileExistsError:
                    pass

    #get latest SEBRA date imported

    imported_dates = []
    for year in years:
        try:
            max_date = max([datetime.strptime(date[:10],'%Y-%m-%d').date() for date in os.listdir(path + '/SEBRA/' + year) if type(re.search('\d{4}-\d{2}-',date)) == re.Match])
            imported_dates.append(max_date)
        except ValueError:
            pass

    # if there are already imported files, re-assign     
    if len(imported_dates) == 0:
        pass
    else:
        max_imported_date = max(imported_dates)
        min_day = max_imported_date.day
        min_month = max_imported_date.month
        min_year = max_imported_date.year

    #starting date to search for data
    start_date = date(min_year,min_month, min_day) #2011-07-22
    end_date = datetime.now().date() #date(2021, 12, 31)
    
    assert start_date <= end_date,'Start date {} is not before end date {}'.format(start_date,end_date)

    print('Path where files will be downloaded: ', path)
    print('Start date selected: {}'.format(start_date))
    print('End date selected: {}'.format(end_date))
    
    #defines the date range: weekends are not included (normally no data is expected here)
    dates =[str(date.date()) for date in pd.date_range(start  = start_date, end = end_date, periods = None, freq='B')]

    ## extract the urls with the relevant dates
    urls = {}
    for d in dates:
        url = 'https://www.minfin.bg/bg/transparency/' + d
        urls[d] = url
    print('All relevant urls to be crawled for the specified period exported')
    
    return urls,dates,start_date,end_date

urls,dates,start_date,end_date = get_urls()


Already existing directory:  /Users/bilyanavencheva/Desktop/All/DG_test/SEBRA
Path where files will be downloaded:  /Users/bilyanavencheva/Desktop/All/DG_test
Start date selected: 2021-05-15
End date selected: 2021-05-27
All relevant urls to be crawled for the specified period exported


In [12]:
%%time

issues_requests = {}
issues_download = {}
issues_rename = {}
soups = {}
processed_files = 0

with requests.Session() as s:
    for d in urls.keys():
        
        
        year = d[:4]
        html = s.get(urls[d])

        if html.status_code != 200:
            issues_requests[d] = 'Status request: ' + str(html.status_code)
        else:
            soups[d] = {'num_files':0,'num_upload':0,'files':[]}
            soup = BeautifulSoup(html.content,'html.parser')

            num_files = 0
            num_upload = 0
            files = []

            for link in soup.find_all('a'):
                #checks and counts if upload is mentioned in the html
                if 'upload' in link.get('href'):
                    num_upload += 1 
                #checks and counts if .xl is mentioned in the html, extracts the links !! to do might need to look for other extensions
                if '.xl' in link.get('href'):
                    num_files += 1
                    files.append('https://www.minfin.bg' + link.get('href'))

            if len(files) > 0:  #if we have any files downloaded
                # check whether the file already exists -> if it does -> replace with new file
                for file in files:
                    
                    # @change files
                    if 'MF' in file or 'NF' in file:  #this line needs to be dropped if we want to download all of them
                        continue   #this line needs to be dropped if we want to download all of them
                    else:  #this line needs to be dropped if we want to download all of them

                        processed_files =+ 1
                        url = file
                        if "NF" in file:
                                location  = path + folders[1] + '/' + year
                        elif "MF" in file:
                                location  = path + folders[2] + '/' + year
                        else:  
                                location  = path + folders[0] + '/' + year

                        file_name = [r.end() for r in re.finditer('/',file)][-1] - 1

                        #if os.path.exists(location + file[last:]) == True:
                            #continue
                        #else:
                        try:
                            # dir(webdriver.common.html5.application_cache.ApplicationCache)
                            customOption = Options()
                            customOption.add_experimental_option('prefs',{'download.default_directory':location})
                            customOption.add_argument("--headless")
                            driver = webdriver.Chrome(executable_path = r'/users/bilyanavencheva/Applications/chromedriver', options = customOption)
                            driver.get(url)
                            time.sleep(5) ## this needs to be addressed
                            driver.close()
                        except Exception as e:
                            issues_download[file] = e

                        ##normalize the date
                        try:
                            os.rename(location + '/'+ file.split('/')[-1],location + '/'+ d + '_' + file.split('/')[-1])
                        except Exception as e:
                            issues_rename[file] = e
            else: 
                continue

            soups[d]['num_files']  = num_files
            soups[d]['num_upload'] = num_upload
            soups[d]['files'] = files

#add new meta data to previous one            
dates_meta = pd.DataFrame.from_dict(soups,orient ='index') #pd.concat([dates_meta,pd.DataFrame.from_dict(soups,orient ='index')])

# files_in_folders = len([file for file in os.listdir(path + folders[0] + '/' + year) if '.xl' in file ]) \
#     + len([file for file in os.listdir(path + folders[1] + '/' + year) if '.xl' in file ]) \
#     + len([file for file in os.listdir(path + folders[2] + '/' + year) if '.xl' in file ])

#summary messages & checks
print('Total number of dates: ', len(dates))
print('Total number of files (incl NF and MF) available: ', sum(dates_meta['num_files']))

#print('Total number of files processed: ', processed_files)
print('Total number of pages checked: ', len(urls))
print('Total number of dates with <> 3 files', len(dates_meta[dates_meta.num_files != 3]))

#requests
if len(issues_requests) == 0:
    print('No issues with url request')
else:
    print('Number of issues url requests: ', len(issues_requests))

#downloads    
if len(issues_download) == 0:
    print('No issues with download of files')
else:
    print('Number of issues with download of files: ', len(issues_download))
    
#rename    
if len(issues_rename) == 0:
    print('No issues with renaming of files')
else:
    print('Number of issues with renaming of files: ', len(issues_rename))

# save pickle file   
name = 'dates_meta_' + str(start_date) + "_" + str(end_date)
with open(path + '/'+ name, 'wb') as config_dictionary_file:
    pickle.dump(dates_meta, config_dictionary_file)

print('Meta data saved in pickled file:',name)

Total number of dates:  9
Total number of files (incl NF and MF) available:  18
Total number of pages checked:  9
Total number of dates with <> 3 files 3
No issues with url request
No issues with download of files
No issues with renaming of files
Meta data saved in pickled file: dates_meta_2021-05-15_2021-05-27
CPU times: user 441 ms, sys: 160 ms, total: 601 ms
Wall time: 1min 2s
